In [1]:
!pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 6.0 MB/s 


In [2]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text
import os
import pandas as pd
import numpy as np

In [3]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [4]:
from google.colab import drive
drive.mount('/content/drive')
INPUT_DIR = "/content/drive/MyDrive/project_data/full_data"
OUTPUT_DIR = "/content/drive/MyDrive/project_data"

Mounted at /content/drive


In [5]:
df_train = pd.read_csv(os.path.join(INPUT_DIR, "train.csv"), header=0)
df_train = df_train.dropna(how="any")
df_train["targetcat_encoded"] = df_train["targetcat"].astype('category').cat.codes
df_train = df_train.rename(columns={"targetcat_encoded": "label", "description": "text"})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
col = 'targetcat'
class MyDict(dict):
    def __missing__(self, key):
        return None
mapping_dict = MyDict({k: v for k, v in zip(df_train['targetcat'], df_train['label'])})
mapping_dict

{'Аксессуары для сауны и бани': 158,
 'Подгузники': 1503,
 'Системы защиты от протечек и залива': 1847,
 'Жалюзи и ролеты': 610,
 'ДСП и ДВП': 479,
 'Щипцы для завивки ресниц': 2408,
 'Карты доступа, активации и онлайн-пополнения пакетов': 797,
 'Автомобильные треки': 65,
 'Мониторы': 1151,
 'Кулеры для воды': 980,
 'Оборудование для бассейнов': 1313,
 'Нагревательные маты': 1237,
 'Мужская парфюмерия': 1186,
 'Наборы инструментов': 1225,
 'Синхронизаторы и ПДУ': 1837,
 'Кухонные вытяжки': 990,
 'Материнские платы': 1083,
 'Телевизоры': 2048,
 'Товары для дома': 2097,
 'МФУ/Принтеры': 1046,
 'Бассейны': 248,
 'Машинки для стрижки': 1086,
 'Зубная паста': 696,
 'Термопродукция': 2073,
 'Портативные колонки': 1550,
 'Аромадиффузоры': 222,
 'Мотошлемы': 1180,
 'Аксессуары и комплектующие для паяльного оборудования': 178,
 'Наушники': 1280,
 'Звукозапись': 685,
 'Роботы-пылесосы': 1717,
 'Компьютеры, неттопы, моноблоки': 895,
 'Тушь для ресниц': 2166,
 'Аксессуары для электронных книг': 17

In [7]:
df_valid = pd.read_csv(os.path.join(INPUT_DIR, "valid.csv"), header=0)
df_valid = df_valid.dropna(how="any")
df_valid["targetcat_encoded"] = df_valid["targetcat"].map(mapping_dict)
df_valid = df_valid.rename(columns={"targetcat_encoded": "label", "description": "text"})
df_valid = df_valid.dropna(how="any")
df_valid["label"] = df_valid["label"].astype(int)

In [8]:
df_train.head()

,text,title,targetcat,label
0,Банная шапка незаменимый аксессуар при посещен...,"Банная шапка Luxyart ""Отаман"", натуральный вой...",Аксессуары для сауны и бани,158
1,Детские подгузники - трусики Helen Harper Baby...,Подгузники-трусики Helen Harper Baby pants Max...,Подгузники,1503
2,Комплект Аквасторож Класик 2*20- это инновацио...,Защита от протечки воды Аквасторож КЛАСИК 2*20,Системы защиты от протечек и залива,1847
3,Роллет на окно Виконда AR39,Роллет на окно Виконда AR39 2100х900 мм (4000000),Жалюзи и ролеты,610
4,"Влагостойкая столешница на основе ДСП, облицо...",Влагостойкая столешница Arpa ДСП 4200x600x40мм...,ДСП и ДВП,479


In [9]:
df_valid[df_valid.label == 1902]

,text,title,targetcat,label
11168,Бурлящий шарик для ванны Кафе Красоты Кофейно-...,Бурлящий шарик для ванны Кафе Красоты Кофейно-...,Соль для ванны,1902
12500,Соль для ванны Кожен День способствует расслаб...,"Соль морская для ванны Кожен День Эвкалипт, 500 г",Соль для ванны,1902
12660,Натуральная соль Мёртвого моря Care & Beauty L...,Натуральная соль Мёртвого моря Care and Beauty...,Соль для ванны,1902
13723,Бомбочки для ванны превратят обычный прием ван...,Бомбочка для ванны Санта 100 г,Соль для ванны,1902
20473,Цветная соль для ванны Nailmatic для детей в у...,Цветная соль для ванны Nailmatic голубая 250 г...,Соль для ванны,1902
31220,Вируюча бомбочка для ванни Склад: бікарбонат н...,Набор бурлящих бомб для ванны ручной работы 6 ...,Соль для ванны,1902
34940,Каждая жемчужина для ванны — это минимальная д...,Жемчужины для ванны Mades Cosmetics Chapter ре...,Соль для ванны,1902
41633,Магниевые ванны – это простой способ побаловат...,Магниевые ванны Бишофит Полтавский BishEffect ...,Соль для ванны,1902
42748,"Принимать ванну будет не скучно, если в Вашем ...",Бомбочка для ванны Beauty Jar Box Of Crayons 1...,Соль для ванны,1902
45291,"Натуральная морская соль расслабляет тело, сни...",Соль для ванны Mr.Scrubber Sia Aphrodisiac (48...,Соль для ванны,1902


In [10]:
hub_layer = hub.KerasLayer(embed, input_shape=[], 
                           dtype=tf.string, trainable=False)

In [11]:
category_counts = len(df_train['label'].value_counts())
category_counts

2463

In [12]:
model = tf.keras.Sequential()
model.add(hub_layer)
#model.add(tf.keras.layers.Dense(128, activation='relu'))
#model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(category_counts, activation='softmax'))
model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               68927232  
                                                                 
 dense (Dense)               (None, 2463)              1263519   
                                                                 
Total params: 70,190,751
Trainable params: 1,263,519
Non-trainable params: 68,927,232
_________________________________________________________________


In [13]:
#module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3'
#embedding_layer = hub.KerasLayer(module_url)
#hidden_layer = tf.keras.layers.Dense(256, activation='relu')
#output_layer = tf.keras.layers.Dense(category_counts, activation='softmax')

#inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
#x = embedding_layer(tf.squeeze(tf.cast(inputs, tf.string)))
#x = hidden_layer(x)
#outputs = output_layer(x)

#model = tf.keras.Model(inputs=inputs, outputs=outputs)

#model.summary()
#model.compile(optimizer='adam',
#              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [14]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(x=df_train['text'],
                    y=df_train['label'],
                    validation_data = (df_valid['text'], df_valid['label']),
                    epochs=25,
                    batch_size=512,
                    callbacks=[early_stopping],
                    verbose=1)

Epoch 1/25
1041/1041 [==============================] - 456s 432ms/step - loss: 5.5132 - sparse_categorical_accuracy: 0.3113 - val_loss: 4.0806 - val_sparse_categorical_accuracy: 0.3927
Epoch 2/25
1041/1041 [==============================] - 453s 435ms/step - loss: 3.3858 - sparse_categorical_accuracy: 0.4650 - val_loss: 2.8874 - val_sparse_categorical_accuracy: 0.5234
Epoch 3/25
1041/1041 [==============================] - 453s 435ms/step - loss: 2.5393 - sparse_categorical_accuracy: 0.5626 - val_loss: 2.3105 - val_sparse_categorical_accuracy: 0.5900
Epoch 4/25
1041/1041 [==============================] - 452s 435ms/step - loss: 2.0878 - sparse_categorical_accuracy: 0.6189 - val_loss: 1.9753 - val_sparse_categorical_accuracy: 0.6328
Epoch 5/25
1041/1041 [==============================] - 457s 439ms/step - loss: 1.8085 - sparse_categorical_accuracy: 0.6549 - val_loss: 1.7585 - val_sparse_categorical_accuracy: 0.6585
Epoch 6/25
1041/1041 [==============================] - 454s 436ms/ste

In [17]:
OUTPUT_DIR = "/content/drive/MyDrive/project_data/keras_model/gt75val"
model.save(OUTPUT_DIR)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/project_data/keras_model/gt75val/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/project_data/keras_model/gt75val/assets


In [ ]:
model.save(os.path.join(OUTPUT_DIR , 'keras_model/my_model.h5'))

In [ ]:
df_test = pd.read_csv(os.path.join(INPUT_DIR, "test.csv"), header=0)
df_test = df_valid.dropna(how="any")
df_test["targetcat_encoded"] = df_test["targetcat"].map(mapping_dict)
model.evaluate(x=df_test['text'], y=df_test['targetcat_encoded'], batch_size=512)

174/174 [==============================] - 66s 379ms/step - loss: 0.9787 - sparse_categorical_accuracy: 0.7693
